# Simulator Integration

In this tutorial we show how to integrate a simulator into OpenSBT. As an example we integrate the [CARLA]() simulator.

OpenSBT offers the abstract class `Simulator`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/simulation/simulator.py) which needs to be implemented to use a concrete simulator. I.e. we need to implement the [simulate]() method of the abstract class. Simulate receives as input a list of scenario instances, and outputs for each scenario instance a `SimulationOutput`. The implementation of `simulate` is simulator specific. Following steps need to be implemented:

1. Passing the scenario instance(s) to the simulator and executing the SUT on the scenario(s).
2. Converting the simulation output returned by the simulator to the format understandable by OpenSBT.


To integrate we CARLA we outsourced the code for step 1 und step 2 in a dedicate module called ```[CarlaRunner](https://git.fortiss.org/fortissimo/ff1_testing/ff1_carla)``` for better maintenance. I.e. sstep 1 is implemented [here]() and step 2 [here]().



The structure of the generic Simulation Output in OpenSBT is defined as follows.

TODO update

```json 
    {
        "simTime" : 3,
        "times": [1.0,2.0,3.0],
        "location": { "ego" : [(1,1),(2,2),(3,3)],
                    "adversary" : [(4,1),(4,2),(4,3)},

        "velocity": { "ego" : [(0.5,0.5,0.5)],
                        "adversary" : [(0.9,0.9,0.9)],
                        },
        "collisions": [],
        "speed" : {
            "ego" : [1,3,3],
            "adversary" : [3,2,5],

         },
        "orientation": {},
        "actors" : {1: "ego",
                        2: "adversary"
                    },
        "otherParams" : {
            "height" : "2",
            "width" : "5
        
        }
    }
```

In the following is the implementation of the **CarlaSimulation** class provided. In line concrete scenario instance are created based on the scenario template provided via **scenario_path** and the generate test inputs in **list_individuals**. 

Then, the discussed CARLA Runner is called from within the simulate method (line X) to execute the SUT in the simulator on concrete scenarios stored in the SCENARIO_DIR folder. To inspect how this execution is triggered please check the [balancer]() class of the CARLA Runner repository. 

The balancer outputs the results via a json file which we can convert into a SimulationOutput instance (line X).

In [4]:
from pathlib import Path
from typing import List, Dict
from simulation.simulator import Simulator, SimulationOutput
import logging as log
import json
import os
from model_ga.individual import Individual
from carla_simulation.balancer import Balancer

SCENARIO_DIR = "/tmp/scenarios"

class CarlaSimulator(Simulator):

    _balancer = None

    ''' Simulates a set of scenarios and returns the simulation traces'''
    @staticmethod
    def simulate(
        list_individuals: List[Individual],
        variable_names: List[str],
        scenario_path: str,
        sim_time: float,
        time_step: float,
        do_visualize: bool = False
    ) -> List[SimulationOutput]:
        xosc = scenario_path
        try:
            for ind in list_individuals:
                instance_values = [v for v in zip(variable_names,ind)]
                CarlaSimulator.create_scenario_instance_xosc(
                            xosc, 
                            dict(instance_values), 
                            outfolder=SCENARIO_DIR)
            if CarlaSimulator._balancer is None:
                CarlaSimulator._balancer = Balancer(
                    directory = SCENARIO_DIR,
                    jobs = 1,
                    visualization = do_visualize,
                    agent = 'NPCAgent'
                )
                CarlaSimulator._balancer.start()
            # Execute SUT on scenario instances
            outs = CarlaSimulator._balancer.run()
            results = []
            for out in outs:
                simout = SimulationOutput.from_json(json.dumps(out))
                results.append(simout)
        except Exception as e:
            raise e
        finally:
            # Remove all created .xosc scenario
            file_list = [ f for f in os.listdir(SCENARIO_DIR) if f.endswith(".xosc") ]
            for f in file_list:
                os.remove(os.path.join(SCENARIO_DIR, f))
        return results

    ''' Replace parameter values in parameter declaration section in generic .xosc file 
    by provided values '''
    @staticmethod
    def create_scenario_instance_xosc(filename: str, values_dict: Dict, outfolder=None):
        import xml.etree.ElementTree as ET
        xml_tree = ET.parse(filename)

        parameters = xml_tree.find('ParameterDeclarations')
        for name, value in values_dict.items():
            for parameter in parameters:
                if parameter.attrib.get("name") == name:
                    parameter.attrib["value"] = str(value)

        # # Write the file out again
        if outfolder is not None:
            Path(outfolder).mkdir(parents=True, exist_ok=True)
            filename = outfolder + os.sep + os.path.split(filename)[1]
        splitFilename =  os.path.splitext(filename)
        newPathPrefix = splitFilename[0]
        ending = splitFilename[1]

        suffix = ""
        for k,v in values_dict.items():
            suffix = suffix + "_" + str(v)

        newFileName  = newPathPrefix + suffix + ending
        xml_tree.write(newFileName)
        return newFileName


ModuleNotFoundError: No module named 'simulation'